In [1]:
using CSV
using DataFrames
using PrettyTables
using StatsBase
include("./advgames.jl")

initialize (generic function with 1 method)

In [2]:
# Define special data to record

adv_coop(player) = player.adv_type == :C
adv_defect(player) = player.adv_type == :D
should_c(player) = est_move_coop(player) > est_move_defect(player)
should_d(player) = est_move_coop(player) < est_move_defect(player)
should_adv_c(player) = est_adv_coop(player) > est_adv_defect(player)
should_adv_d(player) = est_adv_coop(player) < est_adv_defect(player)

function cum_avg(a::Vector{Float64})  # helper function that returns vector of cumulative averages of vector a
      ca=Float64[]    
      for i in 1:length(a)
          push!( ca, sum(first(a,i))/i )
      end
return ca
end


#individual_incls(model) = [pr_coop(p) for p in allagents(model)]
avg_incl(model) = sum([pr_coop(p) for p in allagents(model)]) / nagents(model)  
 
pr_max_coop(model) = sum([pr_coop(p) == 1.0 for p in allagents(model)]) / nagents(model)
pr_max_defect(model) = sum([pr_coop(p) == 0.0 for p in allagents(model)]) / nagents(model)

cooperate_state(model) = pr_max_coop(model)>.75
defect_state(model) = pr_max_defect(model) >.75

pr_adv_coop(model) = sum([advocate(p) == :C for p in allagents(model)]) / nagents(model)
pr_adv_defect(model) = sum([advocate(p) == :D for p in allagents(model)]) / nagents(model)

pr_hyp_adv_c_play_d(model) = sum([advocate(p) == :C && pr_coop(p) <= 0.15 for p in allagents(model)]) / nagents(model)
pr_hyp_adv_d_play_c(model) = sum([advocate(p) == :D && pr_coop(p) >= 0.85 for p in allagents(model)]) / nagents(model)

avg_pr_max_coop(model) = mean([pr_coop(p) for p in allagents(model)])  >= 0.85
avg_pr_max_defect(model) = mean([pr_coop(p) for p in allagents(model)])  <= 0.15

pr_mistaken_adv(model) = sum([(advocate(p) == :C && est_move_coop(p) < est_move_defect(p)) ||
                              (advocate(p) == :D && est_move_coop(p) > est_move_defect(p))
                            for p in allagents(model)]) / nagents(model)

proportion_should_c(model) = sum([should_c(p) for p in allagents(model)]) / nagents(model)
proportion_should_d(model) = sum([should_d(p) for p in allagents(model)]) / nagents(model)
proportion_should_adv_c(model) = sum([should_adv_c(p) for p in allagents(model)]) / nagents(model)
proportion_should_adv_d(model) = sum([should_adv_d(p) for p in allagents(model)]) / nagents(model)
cc(model) = float(model.payouts[:C, :C])
cd(model) = float(model.payouts[:C, :D])
dc(model) = float(model.payouts[:D, :C])
dd(model) = float(model.payouts[:D, :D])

is_pd(model) = dc(model) > cc(model) > dd(model) > cd(model)
is_chicken(model) = dc(model) > cc(model) && cd(model) > dd(model)
is_sh(model) = cc(model) > dc(model) && dd(model) > cd(model) && cc(model) > dd(model)
is_c_dom(model) = cc(model) > dc(model) && cd(model) > dd(model) && cc(model) > dd(model)
is_d_dom(model) = dd(model) > cd(model) && dc(model) > cc(model) && dd(model) > cc(model)

function game_type(model) 
    if is_pd(model)
        return "PD"
    elseif is_chicken(model)
        return "CH"
    elseif is_sh(model)
        return "SH"
    elseif is_c_dom(model)
        return "Cdom"
    elseif is_d_dom(model)
        return "Ddom"
    else 
        return "Other"
    end
end

cooperative(player) = pr_coop(player) >= (1-player.move_increment)
uncooperative(player) = pr_coop(player) <= player.move_increment

perc_cooperative(model) = sum([cooperative(p) for p in allagents(model)]) / nagents(model)

perc_uncooperative(model) = sum([uncooperative(p) for p in allagents(model)]) / nagents(model)

perc_middle(model) = sum([!cooperative(p) && !uncooperative(p) for p in allagents(model)]) / nagents(model)

variance_inclinations(model) = var([pr_coop(p) for p in allagents(model)])



variance_inclinations (generic function with 1 method)

In [3]:
function run_simulation(df, num_simulations, numrounds, params_for_model, params_filename)
    for nsim in 1:num_simulations
        adata = [pr_coop]
        # model data to record
        mdata = [
            cc, 
            cd, 
            dc, 
            dd, 
            game_type,
            avg_incl,
            pr_adv_coop,
            pr_hyp_adv_c_play_d, 
            pr_hyp_adv_d_play_c, 
            perc_cooperative,
            perc_uncooperative,
            perc_middle,
            variance_inclinations
            ]
        # params is a dictionary with the parameters to be used in the simulation
        model, parameters = initialize(; params_for_model...)            
        #pretty_table(parameters, show_subheader=false)
        # write the parameters to a txt file
        open("$(params_filename)_$(nsim).txt", "w") do io
            pretty_table(io, parameters, show_subheader=false)
        end

        _, mdf = run!(model, dummystep, model_step!, numrounds; adata, mdata)
        
        # average the mdf avg_incl
        cai = mean(mdf.avg_incl)

        # proportion of the times perc_cooperative >= 0.75
        time_on_top = sum(mdf.perc_cooperative .>= 0.75) / length(mdf.perc_cooperative) 
        time_on_bottom = sum(mdf.perc_uncooperative .>= 0.75) / length(mdf.perc_cooperative) 

        if time_on_bottom == 0
            coop_degree = Inf
        else
            coop_degree = time_on_top / time_on_bottom
        end
        
        order_degree = time_on_top + time_on_bottom

        R = model.base_payouts[:C, :C]
        S = model.base_payouts[:C, :D]
        T = model.base_payouts[:D, :C]
        P = model.base_payouts[:D, :D]
        tolerance = parameters[:tolerance]
        num_defectors = parameters[:num_defectors]
        num_cooperators = parameters[:num_cooperators]
        num_neutral = parameters[:num_neutral]
        max_positive_pressure = parameters[:max_pos_neg_pressures][1]
        max_negative_pressure = parameters[:max_pos_neg_pressures][2]
        adv_mutation_rate = parameters[:a_m_mutation_rates][1]
        move_mutation_rate = parameters[:a_m_mutation_rates][2]
        weight_of_present_for_adv = parameters[:a_m_weights_of_present][1]
        weight_of_present_for_move = parameters[:a_m_weights_of_present][2]
        adv_reassess_frequency = parameters[:a_m_reassess_frequencies][1]
        move_reassess_frequency = parameters[:a_m_reassess_frequencies][2]
        move_increment = parameters[:m_increment]
        push!(df, [cai, coop_degree, order_degree, nsim, numrounds, R, S, T, P, tolerance, num_defectors, num_cooperators, num_neutral, max_positive_pressure, max_negative_pressure, adv_mutation_rate, move_mutation_rate, weight_of_present_for_adv, weight_of_present_for_move, adv_reassess_frequency, move_reassess_frequency, move_increment])
    end
end


run_simulation (generic function with 1 method)

In [ ]:
params_for_model = Dict(
    :num_cooperators => 10, 
    :num_defectors => 10, 
    :num_neutral => 0, 
    :max_pos_pressure => 6,
    :max_neg_pressure => 4,
    :adv_mutation_rate => 0.2,
    :move_mutation_rate => 0.15,
    :tolerance => 0.05,
    :payouts => Dict(
        (:C, :C) => 3.0,
        (:C, :D) => 0.0,
        (:D, :C) => 4.0,
        (:D, :D) => 1.0
    )
)

df = DataFrame(
    cai=Float64[],
    coop_degree=Float64[], 
    order_degree=Float64[],
    numsim = Int[],
    numrounds=Int[], 
    R=Float64[], 
    S=Float64[], 
    T=Float64[], 
    P=Float64[], 
    tolerance=Float64[], 
    num_defectors=Int[], 
    num_cooperators=Int[], 
    num_neutral=Int[], 
    max_positive_pressure=Float64[], 
    max_negative_pressure=Float64[], 
    adv_mutation_rate=Float64[], 
    move_mutation_rate=Float64[], 
    weight_of_present_for_adv=Float64[], 
    weight_of_present_for_move=Float64[], 
    adv_reassess_frequency=Float64[], 
    move_reassess_frequency=Float64[], 
    move_increment=Float64[]
)
    
run_simulation(df, 10, 50_000, params_for_model, "./test_params")

df

In [8]:
punishments = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0]
rewards = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0]
numrounds = 500_000
num_simulations = 50
df = DataFrame(
    cai=Float64[],
    coop_degree=Float64[], 
    order_degree=Float64[], 
    numsim = Int[],
    numrounds=Int[],
    R=Float64[], 
    S=Float64[], 
    T=Float64[], 
    P=Float64[], 
    tolerance=Float64[], 
    num_defectors=Int[], 
    num_cooperators=Int[], 
    num_neutral=Int[], 
    max_positive_pressure=Float64[], 
    max_negative_pressure=Float64[], 
    adv_mutation_rate=Float64[], 
    move_mutation_rate=Float64[], 
    weight_of_present_for_adv=Float64[], 
    weight_of_present_for_move=Float64[], 
    adv_reassess_frequency=Float64[], 
    move_reassess_frequency=Float64[], 
    move_increment=Float64[]
)

for punishment in punishments
    for reward in rewards

        println("Running simulation for punishment: $punishment and reward: $reward")

        params_for_model = Dict(
            :num_cooperators => 6, 
            :num_defectors => 6, 
            :num_neutral => 0, 
            :max_pos_pressure => 6,
            :max_neg_pressure => 4,
            :adv_mutation_rate => 0.2,
            :move_mutation_rate => 0.15,
            :tolerance => 0.05,
            :payouts => Dict(
                (:C, :C) => reward,
                (:C, :D) => 0.0,
                (:D, :C) => 10.0,
                (:D, :D) => punishment
                )
        )
        run_simulation(df, num_simulations, numrounds, params_for_model, "./diff_payoffs2/params_$(reward)_$(punishment)")
        
        CSV.write("diff_payoffs2/diff_rewards_punishments.csv", df)

    end
end


Running simulation for punishment: 1.0 and reward: 1.0
Running simulation for punishment: 1.0 and reward: 2.0
Running simulation for punishment: 1.0 and reward: 3.0
Running simulation for punishment: 1.0 and reward: 4.0
Running simulation for punishment: 1.0 and reward: 5.0
Running simulation for punishment: 1.0 and reward: 6.0
Running simulation for punishment: 1.0 and reward: 7.0
Running simulation for punishment: 1.0 and reward: 8.0
Running simulation for punishment: 1.0 and reward: 9.0
Running simulation for punishment: 1.0 and reward: 10.0
Running simulation for punishment: 2.0 and reward: 1.0
Running simulation for punishment: 2.0 and reward: 2.0
Running simulation for punishment: 2.0 and reward: 3.0
Running simulation for punishment: 2.0 and reward: 4.0
Running simulation for punishment: 2.0 and reward: 5.0
Running simulation for punishment: 2.0 and reward: 6.0
Running simulation for punishment: 2.0 and reward: 7.0
Running simulation for punishment: 2.0 and reward: 8.0
Running s

In [7]:
triv = Dict(
    (:C, :C) => 0.0,
    (:C, :D) => 0.0,
    (:D, :C) => 0.0,
    (:D, :D) => 0.0
    )

coop = Dict(
    (:C, :C) => 1.0,
    (:C, :D) => 0.0,
    (:D, :C) => 0.0,
    (:D, :D) => 1.0
    )

hilo = Dict(
    (:C, :C) => 3.0,
    (:C, :D) => 0.0,
    (:D, :C) => 0.0,
    (:D, :D) => 1.0
    )

take_turns = Dict(
        (:C, :C) => 1.0,
        (:C, :D) => 2.0,
        (:D, :C) => 3.0,
        (:D, :D) => 0.0
)

chicken = Dict(
        (:C, :C) => 3.0,
        (:C, :D) => 2.0,
        (:D, :C) => 4.0,
        (:D, :D) => 0.0
)

sh = Dict(
        (:C, :C) => 3.0,
        (:C, :D) => 0.0,
        (:D, :C) => 2.0,
        (:D, :D) => 1.0
)

pd = Dict(
        (:C, :C) => 3.0,
        (:C, :D) => 0.0,
        (:D, :C) => 4.0,
        (:D, :D) => 1.0
        )

games = Dict(
    triv => "triv", 
    coop => "coord",
    hilo => "hilo",
    take_turns => "take_turns",
    sh => "sh",
    chicken => "chicken",
    pd => "pd"
)
numrounds = 500_000
num_simulations = 50
df = DataFrame(
    cai=Float64[],
    coop_degree=Float64[], 
    order_degree=Float64[], 
    numsim = Int[],
    numrounds=Int[],
    R=Float64[], 
    S=Float64[], 
    T=Float64[], 
    P=Float64[], 
    tolerance=Float64[], 
    num_defectors=Int[], 
    num_cooperators=Int[], 
    num_neutral=Int[], 
    max_positive_pressure=Float64[], 
    max_negative_pressure=Float64[], 
    adv_mutation_rate=Float64[], 
    move_mutation_rate=Float64[], 
    weight_of_present_for_adv=Float64[], 
    weight_of_present_for_move=Float64[], 
    adv_reassess_frequency=Float64[], 
    move_reassess_frequency=Float64[], 
    move_increment=Float64[]
)

for (gm, gm_name) in games
    println(gm_name)
    params_for_model = Dict(
        :num_cooperators => 6, 
        :num_defectors => 6, 
        :num_neutral => 0, 
        :max_pos_pressure => 6,
        :max_neg_pressure => 4,
        :adv_mutation_rate => 0.2,
        :move_mutation_rate => 0.15,
        :tolerance => 0.05,
        :payouts => gm
    )
    run_simulation(df, num_simulations, numrounds, params_for_model, "./diff_base_games/params_$(gm_name)")
    CSV.write("diff_base_games/diff_base_games.csv", df)

end


triv
hilo
coord
sh
pd
chicken
take_turns
